In [ ]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

In [ ]:
data_df = pd.read_parquet("../dataset/validation_dataset.parquet")
# data_df = data_df[data_df["language"] == "English"]

In [ ]:
subset_df = data_df.sample(100)

In [ ]:
def generate_gpt_label(sentence):
        
    template = f"""Given the following news, output 0 if the ESG impact duration is below 2 years, 1 if the ESG impact duration is between 2 and 5 year and 2 if the ESG impact duration is more than 5 years. You only need to output the number, and do not need any further explanation.
    
    {sentence}"""
    
    conversation = [
            {"role": "system", "content": "You are an ESG analyst, skilled assessing the level and duration an event in the news article might have on the company"},
            {"role": "user", "content": template}
        ]
    
    completion = client.chat.completions.create(
        model="gpt-4",
        messages=conversation,
        temperature=0.7
    )
    
    message = completion.choices[0].message.content
    
    return int(message)


subset_df["gpt_label"] = subset_df["feature"].apply(generate_gpt_label)

In [ ]:
len(subset_df[subset_df["gpt_label"] == subset_df["label"]])

In [ ]:
subset_df.groupby(["gpt_label", "label"])["gpt_label"].count().unstack()